In [ ]:
import numpy as np
import pandas as pd

# Data Manipulation

Since our data set comes in this format, we will go through our game_moves and then return an array of moves.

game_data = {
    'eOthello_game_id': 1056798,
    'winner': 1,
    'game_moves': 'f5d6c4d3e6f4e3f6c5b4e7f3c6d7b5a5c3b3g5h5g4h4e2g6b6d8c7c8a4a6a7f1a3c2d2b2e1b7g3h3f2d1a1a2b1a8c1g1f7g8'
}

We are going to convert the game moves into game states for player 1 and 2

In [ ]:
import numpy as np

# Define the initial board state
initial_board = np.zeros((8, 8), dtype=int)

def update_board(board, move, player):
    # Convert the move string to row and column indices
    col = ord(move[0]) - ord('a')
    row = int(move[1]) - 1

    # Update the board with the player's move
    board[row, col] = player

    return board

def convert_moves_to_board_states(moves):
    # Create lists to store board states for each player
    player1_states = [np.copy(initial_board)]
    player2_states = [np.copy(initial_board)]

    current_player = 1  # Player 1 starts

    for move in moves:
        # Update the board with the current player's move
        if current_player == 1:
            board = update_board(player1_states[-1], move, current_player)
            player1_states.append(np.copy(board))
        else:
            board = update_board(player2_states[-1], move, current_player)
            player2_states.append(np.copy(board))

        # Switch to the next player
        current_player = 3 - current_player  # Alternates between 1 and 2

    return player1_states, player2_states


In [ ]:
# Extracts moves from a string into individual moves
def extract_moves(moves):
    move_list = [moves[i:i+2] for i in range(0, len(moves), 2)]
    player1_states, player2_states = convert_moves_to_board_states(move_list)

    result = {
        'Log Player1': player1_states,
        'Log Player2': player2_states
    }
    return pd.Series(result)

In [ ]:
historic_game_data = pd.read_csv('othello_dataset.csv', header=0,
                                 names=['eOthello Game ID', 'Winner', 'Log'],
                                 converters={'Log': extract_moves},
                                 index_col=['eOthello Game ID'])

# Expand the dataframe with Log Player1 and Log Player2 columns
historic_game_data = pd.concat([historic_game_data['Winner'],
                                historic_game_data['Log'].apply(pd.Series)], axis=1)

## Model for Player 1 Log


In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [ ]:
# Define the input shape
input_shape = (8, 8)

# Create the model
modelBlackPlayer1 = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=input_shape),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])

Divide training and validation data for Black Player 1.

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(historic_game_data['Log Player1'], historic_game_data['Winner'], test_size=0.2, random_state=42)

modelBlackPlayer1.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

In [131]:
# Reshape the input data to have a consistent shape
x_train_reshaped = np.array([np.array(x) for x in x_train])  # Convert each element to a NumPy array
x_train_reshaped = np.reshape(x_train_reshaped, (-1, 8, 8))

# Split the data into training and validation sets
x_train_final, x_val_final, y_train, y_val = train_test_split(x_train_reshaped, y_train, test_size=0.2, random_state=42)

# Convert the NumPy arrays to TensorFlow tensors
tensor_x_train = tf.convert_to_tensor(x_train_final)
tensor_y_train = tf.convert_to_tensor(y_train)


modelBlackPlayer1.fit(tensor_x_train, tensor_y_train, epochs=10, batch_size=32)

C:\Users\afmo2\AppData\Local\Temp\ipykernel_30452\3159464968.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  x_train_reshaped = np.array([np.array(x) for x in x_train])  # Convert each element to a NumPy array


ValueError: cannot reshape array of size 20525 into shape (8,8)